# AILib Tutorial 11: Simplified API - Vercel AI SDK Style

This tutorial introduces AILib's new simplified API, inspired by Vercel AI SDK. Learn how to:

- Use factory functions for quick setup
- Build agents and chains with minimal code
- Understand the philosophy of progressive disclosure
- Migrate from the old verbose API

## The Philosophy: Simple by Default

AILib now offers two ways to create objects:

1. **Factory Functions** (Recommended): Simple, validated, and safe
2. **Direct Instantiation**: More control, no validation

Let's see the difference:

In [ ]:
from ailib import create_agent, create_chain, create_session
from ailib import Agent, Chain, Session  # Direct classes
import os

# Make sure you have your API key set
# os.environ['OPENAI_API_KEY'] = 'your-key-here'

## Creating Agents - The New Way

In [ ]:
# Old way (verbose - LangChain style)
# from ailib import OpenAIClient, Agent
# client = OpenAIClient(model="gpt-4")
# agent = Agent(llm=client, max_steps=10, verbose=True)

# New way (simple - Vercel AI SDK style)
agent = create_agent("assistant", model="gpt-4", verbose=True)

# That's it! The factory function handles:
# - Creating the LLM client
# - Setting up defaults
# - Validating parameters

result = agent.run("What is 2 + 2?")
print(result)

## Adding Tools - Clean and Simple

In [ ]:
from ailib import tool

# Define tools with the decorator
@tool
def calculate(expression: str) -> float:
    """Evaluate a mathematical expression."""
    try:
        return float(eval(expression))
    except Exception as e:
        return f"Error: {e}"

@tool
def get_weather(city: str) -> str:
    """Get the weather for a city."""
    # Mock implementation
    return f"The weather in {city} is sunny and 72°F"

# Create agent with tools in one line
agent_with_tools = create_agent(
    "weather_assistant",
    tools=[calculate, get_weather],
    instructions="You are a helpful assistant that can check weather and do math."
)

# Test it
result = agent_with_tools.run(
    "What's the weather in Paris? Also, what's 15% of 250?"
)
print(result)

## Creating Chains - No More Boilerplate

In [ ]:
# Old way: Create client, then chain, then add steps...
# client = OpenAIClient()
# chain = Chain(client)
# chain.add_system("You are a translator")
# chain.add_user("Translate to Spanish: {text}")

# New way: Just list your prompts!
translation_chain = create_chain(
    "You are a professional translator.",
    "Translate this to Spanish: {text}",
    "Now make it more formal",
    "Finally, add a polite greeting at the beginning"
)

result = translation_chain.run(text="Hello, how are you today?")
print(result)

## Session Management - With Validation

In [ ]:
# Create a session with validation
session = create_session(
    session_id="demo-session",
    max_messages=50,
    metadata={"user": "demo", "purpose": "tutorial"}
)

# Add messages
session.add_system_message("You are a helpful tutor")
session.add_user_message("Explain quantum computing")

# Use with a client
from ailib import OpenAIClient
client = OpenAIClient()

response = client.complete(session.get_messages())
session.add_assistant_message(response.content)

print(f"Session has {len(session)} messages")
print(f"Session ID: {session.session_id}")

## Advanced Options Still Available

The simplified API doesn't limit you - all options are available as kwargs:

In [ ]:
# Create a fully customized agent
advanced_agent = create_agent(
    "advanced",
    model="gpt-4",
    temperature=0.2,
    max_steps=20,
    verbose=True,
    memory_size=50,
    return_intermediate_steps=True,
    api_key=os.environ.get("OPENAI_API_KEY"),  # Can override
    instructions="""You are an expert researcher. 
    Always cite your sources and think step by step."""
)

print(f"Agent configured with temperature: {advanced_agent.temperature}")
print(f"Max steps: {advanced_agent.max_steps}")

## Direct Instantiation for Full Control

When you need to bypass validation or have special requirements:

In [ ]:
# Direct instantiation - no validation!
from ailib import Agent, OpenAIClient

# This allows "invalid" values
custom_agent = Agent(
    llm=OpenAIClient(),
    temperature=5.0,  # Normally limited to 0-2
    max_steps=1000    # Very high!
)

print(f"Agent with temperature {custom_agent.temperature} created!")
print("Note: Use direct instantiation carefully!")

## Comparison: Old vs New

In [ ]:
# Let's see the difference in code length and complexity

print("=== OLD WAY (LangChain style) ===")
print('''
from ailib import OpenAIClient, Agent, Chain
from ailib.validation import AgentConfig, ChainConfig

# Create client
client = OpenAIClient(model="gpt-4", temperature=0.7)

# Create config
config = AgentConfig(
    name="assistant",
    model="gpt-4",
    max_iterations=10,
    temperature=0.7
)

# Create agent
agent = Agent(llm=client, config=config)

# Create chain
chain = Chain(llm=client)
chain.add_system("You are helpful")
chain.add_user("Hello")
''')

print("\n=== NEW WAY (Vercel AI SDK style) ===")
print('''
from ailib import create_agent, create_chain

# Create agent
agent = create_agent("assistant", model="gpt-4")

# Create chain
chain = create_chain("You are helpful", "Hello")
''')

print("\n✨ Much simpler and cleaner!")

## Real Example: Building a Research Assistant

In [ ]:
# Let's build something real with the new API

@tool
def search_papers(query: str, limit: int = 5) -> str:
    """Search for academic papers on a topic."""
    # Mock implementation
    papers = [
        f"Paper {i+1}: Study on {query} - Published 2024"
        for i in range(limit)
    ]
    return "\n".join(papers)

@tool
def summarize_text(text: str, max_words: int = 100) -> str:
    """Summarize text to a specific length."""
    words = text.split()[:max_words]
    return " ".join(words) + "..."

@tool
def save_research(title: str, content: str) -> str:
    """Save research findings to a file."""
    # Mock implementation
    return f"Research '{title}' saved successfully!"

# Create a research assistant in one line!
research_assistant = create_agent(
    "researcher",
    model="gpt-4",
    tools=[search_papers, summarize_text, save_research],
    instructions="""You are an academic research assistant.
    When asked about a topic:
    1. Search for relevant papers
    2. Summarize key findings
    3. Save important research
    Always provide citations and be thorough.""",
    verbose=True
)

# Use it
result = research_assistant.run(
    "Research the latest developments in quantum computing"
)
print(result)

## Migration Guide: Updating Your Code

In [ ]:
# If you have existing code, here's how to migrate:

# OLD CODE:
# from ailib import OpenAIClient, Agent
# client = OpenAIClient(model="gpt-4")
# agent = Agent(llm=client, verbose=True)

# NEW CODE:
from ailib import create_agent
agent = create_agent("my_agent", model="gpt-4", verbose=True)

# The functionality is the same, just simpler!

# For chains:
# OLD: chain = Chain(client).add_user("Hello")
# NEW: 
chain = create_chain("Hello")

# For sessions:
# OLD: session = Session(session_id="abc")
# NEW:
session = create_session(session_id="abc")

print("✅ Migration is easy - just use the create_* functions!")

## Summary

The new simplified API makes AILib much easier to use:

- ✅ **Factory functions** for quick setup
- ✅ **No boilerplate** - just the essentials
- ✅ **Validation built-in** for safety
- ✅ **All options available** through kwargs
- ✅ **Direct instantiation** when you need control

This follows the Vercel AI SDK philosophy:
- Simple by default
- Progressive disclosure of complexity
- Power when you need it

## Next Steps

- Try the [Safety and Moderation](12_safety_and_moderation.ipynb) features
- Explore [Tracing and Debugging](13_tracing_and_debugging.ipynb)
- Build something awesome with the simplified API!

Happy coding! 🚀